# Revisión de código para 2.2 One-sided Jacobi numerical aproximación

**Fecha:** 13 de Abril de 2020

**Responsable de revisión:** León Garay (Test 1 al 4) y Dorely Morales (Test 5 y 6)

**Código revisado**

In [1]:
indices <- function(n) {
  # Crea una lista de tamaño (n-1)n/2 con pares de índices de la siguiente
  #  manera: (1,2),..,(1,n),(2,3),..,(2,n),...,(n-1,n)
  # Args: 
  #    n: número entero postivo 
  #       se refiere al número de columnas
  #Returns:
  #    lista con pares de índices
    a <- NULL
    b <- NULL
    indices <- NULL
    for (i in 1:(n-1)){
    a <- append(a,rep(i,n-i))
    b <- append(b,seq(i+1,n))    
    }
    for(i in 1:round(n*(n-1)/2))
    indices[[i]] <- list(c(a[i], b[i]))
    indices
}

In [2]:
ortogonal <- function(u,v,TOL=10^-8){
  # Verifica si dos vectores son ortogonales, arrojando un 1 si lo es, y un 0 si no lo es.
  # Args: 
    # u, v como vectores de la misma dimensión.Y un valor real de tolerancia TOL(10^-8).
    # Nota: Se sugiere una TOL mayor a 10^-32.
  # Returns: 
    # Valor booleano 0 (no son ortongoales), 1 (son ortogonales)
    if ( norm(u,type ="2") < TOL | norm(v,type ="2") < TOL){
        ret<-0
    } else{ 
        
        if( (u%*%v)  /(norm(u,type ="2")*norm(v,type ="2")) < TOL   ){
    ret<-1
  }
  else{
    ret<-0
  }       
    }
  ret
}

In [3]:
signo<-function(x) {
  # Indica el signo de un número x
  # Args: 
  #    x (numeric): número a revisar
  # Returns:
  #    1 si el número es positivo o cero
  #    -1 si el número es negativo
  
  ifelse(x<0,-1,1)
  }

In [4]:
solver <- function(U,S,V,b){
    # Construye la solución de un sistema de ecuaciones a partir de matrices 
    # U, S, V, y vector b. Se asume que S es diagonal. 
    # Para ello resuelve S d = U^Tb, para construir x=Vd.
    # Notas:
    # 1) Se utilizó la función backsolve para resolver el sistema triangular.
    # 2) Al ser S diagonal, es indistinto si usar un solver para matrices traingulares inferiores o superiores.
    # Args: 
    #  	    U (mxm),V(nxn), S(mxn) matriz diagonal y b (m) un vector.
    # Returns: 
    #      x vector (m)
  d = backsolve(S, t(U)%*%b)
  x = V%*%d
  return(x)
}

In [5]:
svd_jacobi_aprox <- function(A,TOL,maxsweep){
    # Función que calcula la descomposición de una matriz A en sus componentes U, S V, 
    # utilizando el método de Jacobi para calcular la factorización SVD.De esta forma 
    # la matriz A queda descompuesta de la siguiente forma: A = U*S*t(V).
    # Args: 
    #    A (matriz): Matriz de entrada (nxm) de números reales a la que se le calculará la descomposición SVD.
    #    TOL (numeric): controla la convergencia del método, siendo un valor real de 10^-8 (sugerido en la nota 3.3.d.SVD)
    #    Nota: Se sugiere una TOL mayor a 10^-32.
    #    maxsweep (numeric): número máximo de sweeps,donde cada sweep consiste de un número máximo(nmax)
    #    de rotaciones; y en cada sweep se ortogonalizan 2 columnas.
    # Returns: 
    #   Lista con 3 elementos, donde el primer elemento representan a las matriz S(mxm) matriz diagonal,el segundo a la matriz U(nxm)
    #   y el tercero y último a la matriz V (mxm).En conjunto estas tres matrices componen la factorización SVD de la matriz de entrada A.
    
    #dimensiones
    n<-dim(A)[2] #numero de columnas
    m<-dim(A)[1] #numero de filas
    nmax<-n*(n-1)/2

    #inicialza valores del ciclo
    ak<-A
    vk<-diag(n)
    sig <- NULL
    uk <- ak
    num_col_ortogonal<-0
    k<-0

    while(k<=maxsweep & num_col_ortogonal<nmax){
    num_col_ortogonal<-0
    ind <- indices(n)
    for(i in 1:nmax){
      col_j<-ak[,ind[[i]][[1]][2]]
      col_i<-ak[,ind[[i]][[1]][1]]
    
      #comprueba ortogonalidad  
      if(ortogonal(col_i,col_j,TOL)==1){
        num_col_ortogonal<-num_col_ortogonal+1
      }
      else{
        #calcula coeficientes de la matriz
        a<-sum(col_i*col_i)
        b<-sum(col_j*col_j)
        c<-col_i%*%col_j
        
        if(c<TOL){break}
        #calcula la rotacion givens que diagonaliza
        epsilon<-(b-a)/(2*c)
        t<-signo(epsilon)/(abs(epsilon)+sqrt(1+epsilon**2))
        cs<-1/sqrt(1+t**2)
        sn<-cs*t
        
        #actualiza las columnas de la matriz ak
        for(l in seq(1,m)){
          temp<-ak[l,ind[[i]][[1]][1]]
          ak[l,ind[[i]][[1]][1]]<-cs*temp-sn*ak[l,ind[[i]][[1]][2]]
          ak[l,ind[[i]][[1]][2]]<-sn*temp+cs*ak[l,ind[[i]][[1]][2]]
        }
        
        #actualiza las columnas de la matriz vk
        for(l in seq(1,n)){
          temp<-vk[l,ind[[i]][[1]][1]]
          vk[l,ind[[i]][[1]][1]]<-cs*temp-sn*vk[l,ind[[i]][[1]][2]]
          vk[l,ind[[i]][[1]][2]]<-sn*temp+cs*vk[l,ind[[i]][[1]][2]]
        
        }
       }
  }
  k<-k+1
 }
    #Obtener sigma
        for(i in 1:n){
        sig<- append(sig,norm(ak[,i],type ="2"))
    }

    #Obtener U
    for(i in 1:n){
        if (sig[i]<TOL){
            uk[,i]<-0  
        } else{
        uk[,i] <- ak[,i]/sig[i]
        }
    }

    # Indices de sigma ordenada en forma decreciente para ordenar V,S,U
    index <- order(sig,decreasing = TRUE)
    vk <- vk[,index]
    S <- diag(sig[index])
    uk <- uk[,index]

    list(S = S, U = uk, V= vk)
 }   

In [6]:
sel_solver<-function(A,b,TOL=10**-8,maxsweep=20){
    #Función resuelve un sistema de ecuaciones lineales (SEL) utilizando la descomposición SVD
    #por medio del método de One-sided Jacobi 
    #El SEL es de la forma Ax=b
    # Args: 
    #    A (float): matriz de incógnitas del SEL
    #    b (float): vector de igualdada del sistema
    #    TOL (numeric): controla la convergencia del método
    #    maxsweep (int): número máximo de sweeps 
    #Returns: x (float): vector solución 

    svd<-svd_jacobi_aprox(A,TOL,maxsweep)
    x<-solver(svd$U,svd$S,svd$V,b)
    x
}

**1.Sobre la documentación del código/de la función**

¿Se encuentran presentes en la implementación los siguientes elementos? Por favor, ingrese explicaciones detalladas.

**a) Descripción concisa y breve de lo que hace el código/la función**

La función de encuentra bien documentada.

**b) Descripción de sus argumentos de entrada, su significado y rango de valores que pueden tomar**

Parcialmente, se sugiere complementar la descripción de los argumentos de entrada especificando que A tiene que ser una matriz de números reales de dimensión nxm. Además, A es una matriz de coeficientes del sistema (en lugar de incógnitas) para SEL. Y hay que corregir un typo en la descripción de b: "#    b (float): vector de igualdad o lado derecho del sistema"

**c) Descripción de los tipos de argumentos de entrada y de salida (por ejemplo, valores enteros, reales, strings, dataframe, matrices, etc)**

Sí se encuentra bien documentado.

**d) Descripción de la salida de la función, su significado y valores/objetos que deben regresar**

Parcialmente, se sugiere complementar la descripción del argumento de salida especificando que x es de tamaño n.

**2. Cumplimiento de objetivos del código/de la función**

**a) ¿El código cumple los objetivos para los que fue diseñado?**

Sí dado que genera un vector x.

**b) ¿La salida de la función genera el valor necesario?**

Parcialmente, es necesario corregir algunos errores que ocurren cuando se introduce una matriz A de mxn con todos sus elementos aij iguales a un mismo valor constante. Adicionalmente, se necesita hacer optimizaciones a la función ya que para matrices A de tamaño mediano no encuentra la SVD.

**3. Pruebas**

Ocupe la presente sección para hacer diseño de pruebas variando los parámetros que recibe el código la función en diferentes rangos para evaluar su comportamiento y/o detectar posibles fallos


**Test 1**

**Objetivo del test 1:** Verificar que la función se comporta correctamente para una matriz nula A de mxn.

**Implementación del test 1:**

In [7]:
c5 <- rep(0,25)
b5 <- c(7,1,33,24,-49)
A5 <- matrix(c5, nrow=5, ncol=5, byrow = T)
c4 <- rep(0,16)
b4 <- c(15,-6,17,-7)
A4 <- matrix(c4, nrow=4, ncol=4, byrow = T)
c3 <- rep(0,9)
b3 <- c(2,9,-5)
A3 <- matrix(c3, nrow=3, ncol=3, byrow = T)
c2 <- rep(0,4)
A2 <- matrix(c2, nrow=2, ncol=2, byrow = T)
b2 <- c(2,9)

TOL<-10**-8
maxsweep<-20

print(A2)
print(A3)
print(A4)
print(A5)
#x_sel1<-sel_solver(A1,b1,TOL = 10**-15,maxsweep=10000)

     [,1] [,2]
[1,]    0    0
[2,]    0    0
     [,1] [,2] [,3]
[1,]    0    0    0
[2,]    0    0    0
[3,]    0    0    0
     [,1] [,2] [,3] [,4]
[1,]    0    0    0    0
[2,]    0    0    0    0
[3,]    0    0    0    0
[4,]    0    0    0    0
     [,1] [,2] [,3] [,4] [,5]
[1,]    0    0    0    0    0
[2,]    0    0    0    0    0
[3,]    0    0    0    0    0
[4,]    0    0    0    0    0
[5,]    0    0    0    0    0


In [8]:
x_sel2 <- sel_solver(A2,b2,TOL,maxsweep)
#x_sel3 <- sel_solver(A3,b3,TOL,maxsweep)
#x_sel4 <- sel_solver(A4,b4,TOL,maxsweep)
#x_sel5 <- sel_solver(A5,b5,TOL,maxsweep)

ERROR: Error in backsolve(S, t(U) %*% b): singular matrix in 'backsolve'. First zero in diagonal [1]


**Principales hallazos del test 1**

La función arroja un error pues existen infinitas soluciones que satisfacen A=U*S*V^T. Esto debido a que si tomamos la matriz S nula de mxn y cualesquiera matrices U de mxm y V de nxn ortogonales tenemos una solución.

**Test 2**

**Objetivo del test 2:** Verificar que la función se comporta correctamente para una matriz no nula A de mxn con entradas constantes.

**Implementación del test 2:**

In [9]:
cte <- 5
c5 <- rep(cte,25)
b5 <- c(7,1,33,24,-49)
A5 <- matrix(c5, nrow=5, ncol=5, byrow = T)
c4 <- rep(cte,16)
b4 <- c(15,-6,17,-7)
A4 <- matrix(c4, nrow=4, ncol=4, byrow = T)
c3 <- rep(cte,9)
b3 <- c(2,9,-5)
A3 <- matrix(c3, nrow=3, ncol=3, byrow = T)
c2 <- rep(cte,4)
A2 <- matrix(c2, nrow=2, ncol=2, byrow = T)
b2 <- c(2,9)

TOL<-10**-8
maxsweep<-20

print(A2)
print(A3)
print(A4)
print(A5)
#x_sel1<-sel_solver(A1,b1,TOL = 10**-15,maxsweep=10000)

     [,1] [,2]
[1,]    5    5
[2,]    5    5
     [,1] [,2] [,3]
[1,]    5    5    5
[2,]    5    5    5
[3,]    5    5    5
     [,1] [,2] [,3] [,4]
[1,]    5    5    5    5
[2,]    5    5    5    5
[3,]    5    5    5    5
[4,]    5    5    5    5
     [,1] [,2] [,3] [,4] [,5]
[1,]    5    5    5    5    5
[2,]    5    5    5    5    5
[3,]    5    5    5    5    5
[4,]    5    5    5    5    5
[5,]    5    5    5    5    5


In [10]:
x_sel2 <- sel_solver(A2,b2,TOL,maxsweep)
#x_sel3 <- sel_solver(A3,b3,TOL,maxsweep)
#x_sel4 <- sel_solver(A4,b4,TOL,maxsweep)
#x_sel5 <- sel_solver(A5,b5,TOL,maxsweep)

ERROR: Error in backsolve(S, t(U) %*% b): singular matrix in 'backsolve'. First zero in diagonal [2]


**Principales hallazos del test 2**

La función arroja un error pues existen infinitas soluciones que satisfacen A=U*S*V^T. Esto debido a que si tomamos la matriz S nula de mxn y cualesquiera matrices U de mxm y V de nxn ortogonales tenemos una solución.

**Test 3**

**Objetivo del test 3:** Verificar que la función se comporta correctamente para una matriz A de nxn con n = 2,3,4,5.

**Implementación del test 3:**

In [11]:
c5 <- c(2,-1,4,1,-1,-1,3,-2,-1,2,5,1,3,-4,1,3,-2,-2,-2,3,-4,-1,-5,3,-4)
b5 <- c(7,1,33,24,-49)
A5 <- matrix(c5, nrow=5, ncol=5, byrow = T)
c4 <- c(1,-2,2,-3,3,4,-1,1,2,-3,2,-1,1,1,-3,-2)
b4 <- c(15,-6,17,-7)
A4 <- matrix(c4, nrow=4, ncol=4, byrow = T)
c3 <- c(2,-1,1,3,1,-2,-1,2,5)
b3 <- c(2,9,-5)
A3 <- matrix(c3, nrow=3, ncol=3, byrow = T)
c2 <- c(2,-3,5,2)
A2 <- matrix(c2, nrow=2, ncol=2, byrow = T)
b2 <- c(2,9)

TOL<-10**-8
maxsweep<-20

print(A2)
print(A3)
print(A4)
print(A5)
#x_sel1<-sel_solver(A1,b1,TOL = 10**-15,maxsweep=10000)

     [,1] [,2]
[1,]    2   -3
[2,]    5    2
     [,1] [,2] [,3]
[1,]    2   -1    1
[2,]    3    1   -2
[3,]   -1    2    5
     [,1] [,2] [,3] [,4]
[1,]    1   -2    2   -3
[2,]    3    4   -1    1
[3,]    2   -3    2   -1
[4,]    1    1   -3   -2
     [,1] [,2] [,3] [,4] [,5]
[1,]    2   -1    4    1   -1
[2,]   -1    3   -2   -1    2
[3,]    5    1    3   -4    1
[4,]    3   -2   -2   -2    3
[5,]   -4   -1   -5    3   -4


In [12]:
x_sel2 <- sel_solver(A2,b2,TOL,maxsweep)
x_sel3 <- sel_solver(A3,b3,TOL,maxsweep)
x_sel4 <- sel_solver(A4,b4,TOL,maxsweep)
x_sel5 <- sel_solver(A5,b5,TOL,maxsweep)

Buscamos que la función nos regrese aproximadamente $b2 = (2,9) $

In [13]:
A2%*%x_sel2

2
9


Podemos ver que funcionó correctamente, ya que Ax=b.

Ahora probaremos con $A_{3x3}$ buscamos $b3 = (2,9,-5)$

In [14]:
A3%*%x_sel3

2.327966
11.280633
-8.629119


En este caso podemos ver que la función no arrojó correctamente los valores de Ax=b.

Seguimos con el caso $A_{4x4}$ y buscamos que $b4 = (15,-6,17,-7)$

In [15]:
A4%*%x_sel4

22.127278
-12.551705
23.677302
-8.309975


Podemos observar que tampoco arrojó balores cercanos de Ax=b.

Finalmente probaremos con $A-{5x5}$ y buscamos $b5 = (7,1,33,24,-49)$

In [16]:
A5%*%x_sel5

1.013013
10.506870
62.822134
39.049742
-84.244898


Podemos observar que tampoco arrojó valores correctos.

**Principales hallazos del test 3**

La función arroja valores lejamos a los esperados para matrices con dimensiones mayores a 2x2.

**Test 4**

**Objetivo del test 4:** Verificar que la función se comporta correctamente para una matriz A de mxn con n = 2,3,4,5 y m = 2,3,4.

**Implementación del test 4:**

In [17]:
c5 <- c(2,-1,4,1,-1,-1,3,-2,-1,2,5,1,3,-4,1,3,-2,-2,-2,3)
b5 <- c(7,1,33,24,-49)
A5 <- matrix(c5, nrow=5, ncol=4, byrow = T)
c4 <- c(1,-2,2,-3,3,4,-1,1,2,-3,2,1)
b4 <- c(15,-6,17,-7)
A4 <- matrix(c4, nrow=4, ncol=3, byrow = T)
c3 <- c(2,-1,1,3,1,-2)
b3 <- c(2,9,-5)
A3 <- matrix(c3, nrow=3, ncol=2, byrow = T)
c2 <- c(2,-3,5,2)
A2 <- matrix(c2, nrow=2, ncol=2, byrow = T)
b2 <- c(2,9)

TOL<-10**-8
maxsweep<-20

print(A2)
print(A3)
print(A4)
print(A5)
#x_sel1<-sel_solver(A1,b1,TOL = 10**-15,maxsweep=10000)

     [,1] [,2]
[1,]    2   -3
[2,]    5    2
     [,1] [,2]
[1,]    2   -1
[2,]    1    3
[3,]    1   -2
     [,1] [,2] [,3]
[1,]    1   -2    2
[2,]   -3    3    4
[3,]   -1    1    2
[4,]   -3    2    1
     [,1] [,2] [,3] [,4]
[1,]    2   -1    4    1
[2,]   -1   -1    3   -2
[3,]   -1    2    5    1
[4,]    3   -4    1    3
[5,]   -2   -2   -2    3


In [18]:
x_sel2 <- sel_solver(A2,b2,TOL,maxsweep)
x_sel3 <- sel_solver(A3,b3,TOL,maxsweep)
x_sel4 <- sel_solver(A4,b4,TOL,maxsweep)
x_sel5 <- sel_solver(A5,b5,TOL,maxsweep)

Buscamos que la función nos regrese aproximadamente $b2 = (2,9) $

In [19]:
A2%*%x_sel2

2
9


Podemos ver que funcionó correctamente, ya que Ax=b.

Ahora probaremos con $A_{3x3}$ buscamos $b3 = (2,9,-5)$

In [20]:
A3%*%x_sel3

0.1666667
8.8333333
-3.6666667


En este caso podemos ver que la función no arrojó correctamente los valores de Ax=b.

Seguimos con el caso $A_{4x4}$ y buscamos que $b4 = (15,-6,17,-7)$

In [21]:
A4%*%x_sel4

19.2029412
-5.6382353
0.5911765
-11.7852941


Podemos observar que tampoco arrojó balores cercanos de Ax=b.

Finalmente probaremos con $A-{5x5}$ y buscamos $b5 = (7,1,33,24,-49)$

In [22]:
A5%*%x_sel5

27.976974
11.503845
18.094858
8.093239
-37.052412


Podemos observar que tampoco arrojó valores esperados.

**Principales hallazos del test 4**

La función arroja valores lejanos a los esperados para matrices con dimensiones mayores a 2x2.


**Test 5**

**Objetivo del test 5:** Verificar que la función se comporta correctamente para matrices A de $10^2 \times 10^2$ **pseudoaleatorias** y un sistema homogéneo.

**Implementación del test 5:**

Para la creación de matrices pseudoaleatorias, haremos uso del paquete **mlsjunkgen** que dado un conjunto de input seeds: w,x,y,z genera matrices de números pseudo-aleatorios entre 0 y 1.

In [7]:
install.packages("mlsjunkgen",lib="/usr/local/lib/R/site-library/",
                repos="https://cran.itam.mx/",verbose=TRUE)

system (cmd0): /usr/lib/R/bin/R CMD INSTALL

foundpkgs: mlsjunkgen, /tmp/RtmpfyICEk/downloaded_packages/mlsjunkgen_0.1.1.tar.gz

files: /tmp/RtmpfyICEk/downloaded_packages/mlsjunkgen_0.1.1.tar.gz

1): succeeded '/usr/lib/R/bin/R CMD INSTALL -l '/usr/local/lib/R/site-library' /tmp/RtmpfyICEk/downloaded_packages/mlsjunkgen_0.1.1.tar.gz'



In [8]:
library(mlsjunkgen)

In [9]:
w <- 1
x <- 5
y <- 3
z <- 4

n<-10**2

#Definimos A matriz pseudo-aleatoria de números entre 0 y 1 con 16 decimales 
A<-mlsjunkgenm(nrow = n, ncol = n, w = w, x = x, y = y, z = z, round = 16)
#Definimos b vector de ceros
b<-c(rep(0, len=n))
TOL<-10**-8
maxsweep<-20

In [10]:
#Función
x_sel <- sel_solver(A,b,TOL,maxsweep)

In [12]:
#Comprobamos que la solución x es correcta
Comp<-A%*%x_sel
all(Comp==b)

[1] TRUE

**Principales hallazos del test 5**

* La función arroja resultados correctos para el SEL propuesto.

**Test 6**

**Objetivo del test 6:** Verificar que la función se comporta correctamente para matrices A de $10^2 \times 10^2$ y $10^1 \times 10^1$ **pseudoaleatorias** y diferentes lados derechos.

**Implementación del test 6:**

In [13]:
w <- 1
x <- 5
y <- 3
z <- 4

n<-10**2

#Definimos A matriz pseudo-aleatoria de números entre 0 y 1 con 16 decimales 
A<-mlsjunkgenm(nrow = n, ncol = n, w = w, x = x, y = y, z = z, round = 16)
TOL<-10**-8
maxsweep<-20
A

0.847093986,0.36168442,0.77330303,0.77242522,0.63557536,0.38933633,0.437719895,0.29639091,0.55680276,0.748826701,⋯,0.105783077,0.99061593,0.84818752,0.37081655,0.86610689,0.099294222,0.42965591,0.04260894,0.12798122,0.340566241
0.746115227,0.73389915,0.55074111,0.86946199,0.86020675,0.46693459,0.860517084,0.90706054,0.39866062,0.054475579,⋯,0.876869208,0.61195819,0.43295866,0.21917499,0.71633592,0.024584666,0.03500316,0.80819424,0.48228823,0.081167505
0.436393491,0.54766020,0.95701550,0.55969012,0.12996766,0.98141020,0.683178602,0.82494290,0.44867648,0.347169291,⋯,0.916785280,0.65797825,0.43886193,0.63009880,0.28394267,0.520462599,0.29779099,0.79734805,0.28453628,0.853584486
0.003548722,0.39197078,0.06625749,0.84554358,0.96037845,0.27967929,0.076055612,0.26137673,0.68498094,0.528052706,⋯,0.381578084,0.23450814,0.73509135,0.33784795,0.43191872,0.480533013,0.65780135,0.05478402,0.99987130,0.565547002
0.262946911,0.05150537,0.75235196,0.85939965,0.70175734,0.21235088,0.946810412,0.81798472,0.43112900,0.788326610,⋯,0.390865956,0.73193649,0.27403011,0.59904680,0.21075771,0.857868188,0.64555033,0.52056738,0.06092601,0.006528081
0.324979464,0.90380362,0.85276315,0.50764604,0.31257446,0.97423442,0.745266893,0.07476735,0.78205285,0.080686493,⋯,0.916048921,0.35475290,0.82374186,0.96604516,0.70459031,0.061150261,0.33830607,0.10863037,0.30356968,0.827696467
0.622638935,0.54941419,0.88189912,0.86039902,0.71774556,0.31272255,0.173267526,0.76588454,0.32562647,0.755235776,⋯,0.511089633,0.81347593,0.12736457,0.40042274,0.50047315,0.264999833,0.38756355,0.19027447,0.75240920,0.571884661
0.118732307,0.68051559,0.76047701,0.48606880,0.99146612,0.63651436,0.431093580,0.23703407,0.98460707,0.411111143,⋯,0.387606692,0.52376974,0.76203318,0.39178065,0.10929500,0.974052620,0.96947347,0.24320695,0.65034552,0.859907216
0.733660081,0.52867955,0.85257220,0.18448507,0.19690196,0.04485164,0.688682007,0.68806613,0.16214976,0.853113471,⋯,0.560976589,0.44064833,0.66679074,0.64832613,0.69730278,0.999826525,0.99216408,0.22286483,0.60342368,0.277724271
0.039080997,0.73732215,0.12979027,0.26085527,0.03380972,0.77457998,0.458568254,0.25638119,0.17124881,0.666538066,⋯,0.201221782,0.52840397,0.37924595,0.80459883,0.67467377,0.483378378,0.77956018,0.53568988,0.44846674,0.588118518
0.971389061,0.93808306,0.25011178,0.62212096,0.96132187,0.59397176,0.125172130,0.86667313,0.37282831,0.818885303,⋯,0.991129069,0.14493684,0.62233704,0.59871229,0.52331029,0.814053416,0.94163583,0.30620256,0.91136249,0.041044260


In [19]:
#Definimos b vector de números aleatorios
set.seed(123)
b<-c(runif(n = n, min = -100, max = 100))

In [15]:
#Función
x_sel <- sel_solver(A,b,TOL,maxsweep)

**Sin embargo, en este caso Comp=Ax es diferente del vector b.**

In [16]:
#Comprobamos si la solución x es correcta
Comp<-A%*%x_sel
all(Comp==b)

[1] FALSE

In [25]:
#Verificamos que ninguno de los elementos de tamaño 100 coinciden
length(which(Comp != b))

[1] 100

**Notamos que los elementos de ambos vectores parecen coincidir respecto a su signo y las diferencias entre cada entrada parecieran mantenerse cercanas.**

In [28]:
head(Comp,10)
head(b,10)

-58.708899
50.904351
-23.058111
80.033017
82.579571
-95.200730
1.827158
78.987237
14.143637
-13.340424


[1] -42.484496  57.661027 -18.204616  76.603481  88.093457 -90.888700
 [7]   5.621098  78.483809  10.287003  -8.677053

**Ahora reduciremos la dimensión de A por $10^1 \times 10^1$**

In [36]:
w <- 1
x <- 5
y <- 3
z <- 4

n<-10**1

#Definimos A matriz pseudo-aleatoria de números entre 0 y 1 con 16 decimales 
A<-mlsjunkgenm(nrow = n, ncol = n, w = w, x = x, y = y, z = z, round = 16)
TOL<-10**-8
maxsweep<-20
A

0.847093986,0.9713891,0.8645104,0.28206127,0.78764684,0.45964341,0.4361059,0.438944685,0.42980199,0.01005201
0.746115227,0.2617650,0.6417098,0.70219179,0.36677052,0.52111346,0.5854412,0.056552321,0.05502361,0.84160455
0.436393491,0.7464926,0.5071930,0.70899268,0.98794839,0.45742096,0.8007255,0.612099773,0.17579378,0.27279581
0.003548722,0.1802593,0.1615858,0.26811666,0.85106955,0.79418897,0.4221502,0.603443788,0.23405934,0.23172384
0.262946911,0.5545452,0.3542665,0.03499987,0.07751327,0.95024571,0.6641080,0.775671734,0.70373065,0.47395619
0.324979464,0.5617152,0.3767844,0.40179265,0.26776372,0.54135361,0.8696630,0.298280690,0.59194178,0.16926407
0.622638935,0.1296170,0.4159080,0.04488852,0.59169089,0.45936268,0.9651222,0.860343546,0.89873455,0.42812996
0.118732307,0.5340533,0.6795494,0.96684736,0.28750470,0.07005899,0.9454620,0.538394625,0.47723770,0.66889826
0.733660081,0.9825798,0.1018399,0.01974215,0.65805039,0.58552912,0.1830504,0.699020421,0.05117424,0.32957337
0.039080997,0.3026420,0.5166428,0.32624773,0.13347826,0.35099209,0.4242837,0.005065657,0.08016398,0.14428638


In [37]:
#Definimos b vector de números aleatorios
set.seed(123)
b<-c(runif(n = n, min = -100, max = 100))

In [38]:
#Función
x_sel <- sel_solver(A,b,TOL,maxsweep)

**Y vemos que para este caso Comp=Ax vuelve a ser diferente del vector b.**

In [39]:
#Comprobamos si la solución x es correcta
Comp<-A%*%x_sel
all(Comp==b)

[1] FALSE

In [40]:
#Verificamos que ninguno de los elementos de tamaño 10 coinciden
length(which(Comp != b))

[1] 10

**Notamos que los elementos de ambos vectores vuelven a coincidir respecto a su signo y las diferencias entre cada entrada se hacen más pequeñas.**

In [42]:
Comp
b

-46.240477
50.092697
-30.286978
69.009522
87.828847
-66.945451
15.322677
92.637391
27.256883
5.875627


[1] -42.484496  57.661027 -18.204616  76.603481  88.093457 -90.888700
 [7]   5.621098  78.483809  10.287003  -8.677053

Con la comprobación, vemos que las entradas de Comp=Ax se acercan más a las del vector de b. Sin embargo, las diferencias siguen siendo relativamente grandes.

In [45]:
#Calculamos el promedio del valor absoluto de las diferencias entre ambos vectores
mean((abs(Comp-b)))

[1] 11.05862

**Por último reduciremos la dimensión de A de $2 \times 2$**

In [46]:
w <- 1
x <- 5
y <- 3
z <- 4

n<-2

#Definimos A matriz pseudo-aleatoria de números entre 0 y 1 con 16 decimales 
A<-mlsjunkgenm(nrow = n, ncol = n, w = w, x = x, y = y, z = z, round = 16)
TOL<-10**-8
maxsweep<-20
A

0.8470940,0.436393491
0.7461152,0.003548722


In [47]:
#Definimos b vector de números aleatorios
set.seed(123)
b<-c(runif(n = n, min = -100, max = 100))

In [48]:
#Función
x_sel <- sel_solver(A,b,TOL,maxsweep)

**Y vemos que para este caso Comp=Ax vuelve a ser diferente del vector b.**

In [49]:
#Comprobamos si la solución x es correcta
Comp<-A%*%x_sel
all(Comp==b)

[1] FALSE

**Sin embargo, la primera entrada de ambos vectores sí coincide**

In [60]:
#Al comparar Comp y b, sí coincide la primera entrada pero la segunda es distinta
which(Comp != b)

[1] 2

In [51]:
Comp
b

-42.48450
57.66103


[1] -42.48450  57.66103

**Bajo un redondeo a 13 cifras, ambos vectores coinciden**

In [58]:
#Volvemos a hacer la comparación y vemos que todas las entradas coinciden a 13 cifras
length(which(round(Comp,13) != round(b,13)))

[1] 0

In [62]:
#Calculamos el promedio del valor absoluto de las diferencias entre ambos vectores
(abs(Comp-b))

0.000000e+00
7.105427e-15


**Principales hallazos del test 6**

* La función parece no aproximar de manera precisa la solución para SEL propuestos con A matrices y lados derechos de números pseudo-aleatorios.

**4. Resumen detallado de posibles puntos faltantes en implementación y Sugerencias para resolver los puntos anteriores**

* Se sugiere complementar la descripción de los argumentos de entrada especificando que A tiene que ser una matriz de números reales de dimensión nxm. Además, mencionar que A es una matriz de coeficientes del sistema (en lugar de incógnitas) para SEL. Y se recomienda complementar la descripción de b: "#    b (float): vector de igualdad o lado derecho del sistema"
* Se sugiere complementar la descripción del argumento de salida especificando que x es de tamaño n.
* Necesita revisarse el caso de matrices con todos sus elementos iguales a una constante pues no encuentra la SVD.
* Es necesario verificar si existen errores de aritmética de máquina al calcular los valores de x que resuelven SEL para matrices y lados derechos con números pseudo-aleatorios pues no se satisfizo la comprobación de que Ax sea igual al valor b propuesto.